# Make personalized HLA reference and mask the original reference

This tutorial walks through how to create personalized FASTA and GTF files for the HLA genes per sample, and how to create a masked version of the GRCh38 reference.

To run these steps, you need to download the GRCh38 reference and Gencode GTF file (we use v38).


Written by Amber Shen & Joyce Kang

Last updated Feb 16, 2023

In [1]:
suppressPackageStartupMessages({
    library(rtracklayer)
    library(Biostrings)
    library(dplyr)
    library(stringr)
    library(tidyverse)
})

## Make personalized FASTA and GTF files

In [2]:
source('make_pers_utils.R') # Functions to make personalized genomes and annotations

Read in IMGT allele database

In [3]:
sequence_database_path = '../1_make_HLA_database/IMGTHLA_all_alleles_FINAL.fa'
database = Biostrings::readDNAStringSet(sequence_database_path)
formatted_database_names = lapply(names(database), format_allele)

Examine the database

In [4]:
database
head(formatted_database_names, 4)
tail(formatted_database_names, 4)

DNAStringSet object of length 26573:
        width seq                                           names               
    [1]  4626 AAGGGGAGAGGAGGGCCTGAA...ATGAGAACCTTCCAGAGTCCA IMGT_A*01:01:01:01
    [2]  4626 AAGGGGAGAGGAGGGCCTGAA...ATGAGAACCTTCCAGAGTCCA IMGT_A*01:01:01:03
    [3]  4626 AAGGGGAGAGGAGGGCCTGAA...ATGAGAACCTTCCAGAGTCCA IMGT_A*01:01:01:04
    [4]  4626 AAGGGGAGAGGAGGGCCTGAA...ATGAGAACCTTCCAGAGTCCA IMGT_A*01:01:01:05
    [5]  4626 AAGGGGAGAGGAGGGCCTGAA...ATGAGAACCTTCCAGAGTCCA IMGT_A*01:01:01:06
    ...   ... ...
[26569] 11548 GCATCCACAGAATCACATTTT...TTTCTCAAAGATGGAGTTAAA IMGT_DRB1*16:66:m...
[26570] 11548 GCATCCACAGAATCACATTTT...TTTCTCAAAGATGGAGTTAAA IMGT_DRB1*16:67:m...
[26571] 11547 GCATCCACAGAATCACATTTT...TTTCTCAAAGATGGAGTTAAA IMGT_DRB1*16:68:m...
[26572] 11548 GCATCCACAGAATCACATTTT...TTTCTCAAAGATGGAGTTAAA IMGT_DRB1*16:69:m...
[26573] 11548 GCATCCACAGAATCACATTTT...TTTCTCAAAGATGGAGTTAAA IMGT_DRB1*16:71:m...

[[1]]
[1] "A-01-01"

[[2]]
[1] "A-01-01"

[[3]]
[1] "A-01-01"

[[4]]
[1] "A-01-01"

[[1]]
[1] "DRB1-16-67"

[[2]]
[1] "DRB1-16-68"

[[3]]
[1] "DRB1-16-69"

[[4]]
[1] "DRB1-16-71"

### Make files for example data

In [5]:
personalized_alleles_path = '../example_data/inputs/alleles' 
out = '../example_outputs/personalized_references/'
if (! dir.exists(out)) { dir.create(out) }
make_references(personalized_alleles_path, sequence_database_path, out)

## Mask the reference genome HLA genes with N's

**Compare Indicies Gene vs. All Annotations** <br />
Make sure indices are the same to chose optimal masking

In [6]:
# Directory to starting files (GRCh38 genome and Gencode annotation files)
dir = '../example_data/GRCh38_refs/'

# File path to write masked genome
genome_out = paste0(dir, '/GRCh38.primary_assembly.genome_maskedHLAClassical.fa')

In [7]:
annotations = rtracklayer::import(paste0(dir, 'gencode.v38.annotation.gtf'))
genome = Biostrings::readDNAStringSet(paste0(dir, 'GRCh38.primary_assembly.genome.fa'))
classical = c('HLA-A', 'HLA-B', 'HLA-C', 'HLA-DRB1', 'HLA-DQA1', 'HLA-DPA1', 'HLA-DPB1', 'HLA-DQB1')
classical_annotations = annotations[which(annotations$gene_name %in% classical),]

In [8]:
rownames = c('HLA-A', 'HLA-B', 'HLA-C', 'HLA-DRB1', 'HLA-DQA1', 'HLA-DPA1', 'HLA-DPB1', 'HLA-DQB1')
colnames = c('gene_start', 'min_start', 'gene_end', 'max_end')
start_end = data.frame(matrix(ncol=length(colnames), nrow=length(rownames), dimnames=list(rownames, colnames)))

for (gene in classical) {   
    all_annotations = classical_annotations[which(classical_annotations$gene_name == gene),]
    start_end[gene, 'min_start'] = min(start(ranges(all_annotations)))
    start_end[gene, 'max_end'] = max(end(ranges(all_annotations)))
    
    gene_annotation = all_annotations[which(all_annotations$type == 'gene')]
    start_end[gene, 'gene_start'] = start(ranges(gene_annotation))
    start_end[gene, 'gene_end'] = end(ranges(gene_annotation))
}

print(start_end$gene_start == start_end$min_start)
print(start_end$gene_end == start_end$max_end)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE


**Mask Classical Gene in GRCh38**

By performing a multiple sequence alignment between the GRCh38 HLA gene (length determined by Gencode v38 annotations) and the IMGT genomic sequences, we noticed that for HLA-B and HLA-C, the IMGT sequences have extra sequence content at the end of the IMGT sequence (corresponding to the 3' end of the GRCh38 sequence since these genes are on the negative strand). Hence, we adjust the start and end locations on GRCh38 to mask the appropriate extra sequence.

In [9]:
# adjust for missing 3'UTR annotation in HLA-B and HLA-C
start_end$adjusted_start = start_end$gene_start
start_end['HLA-B', 'adjusted_start'] = start_end['HLA-B', 'adjusted_start'] - 510 # constant determined by MSA
start_end['HLA-C', 'adjusted_start'] = start_end['HLA-C', 'adjusted_start'] - 495 # constant determined by MSA
start_end

,gene_start,min_start,gene_end,max_end,adjusted_start
,<int>,<int>,<int>,<int>,<dbl>
HLA-A,29941260,29941260,29945884,29945884,29941260
HLA-B,31353872,31353872,31357188,31357188,31353362
HLA-C,31268749,31268749,31272130,31272130,31268254
HLA-DRB1,32578769,32578769,32589848,32589848,32578769
HLA-DQA1,32628179,32628179,32647062,32647062,32628179
HLA-DPA1,33064569,33064569,33080775,33080775,33064569
HLA-DPB1,33075990,33075990,33089696,33089696,33075990
HLA-DQB1,32659467,32659467,32668383,32668383,32659467


In [10]:
# Get chr 6 sequence
chr6 = genome['chr6 6'][[1]] # all annotations point to chr6 so this is okay

for (gene in classical) {
    start = start_end[gene, 'adjusted_start']
    end = start_end[gene, 'gene_end']
    mask = paste(rep('N', nchar(as.character(chr6[start:end]))), collapse="")
    chr6[start:end] = as(mask, "DNAString")
}

genome['chr6 6'][[1]] = chr6
Biostrings::writeXStringSet(genome, genome_out) # save file
genome_masked = Biostrings::readDNAStringSet(genome_out)

In [11]:
# Check to make sure masking worked
genome_masked = Biostrings::readDNAStringSet(genome_out)

chr6 = genome['chr6 6'][[1]]
for (gene in classical) {
    start = start_end[gene, 'adjusted_start']
    end = start_end[gene, 'gene_end']
    print(as.character(chr6[start:end]) == paste(rep('N', nchar(as.character(chr6[start:end]))), collapse=""))
}

[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE


## Remove classical HLA entries from reference GTF file

**Make Annotation with Classical Removed** <br />
STARsolo does not like annotations pointing to 'NNNN...'

Note: We filtered the original gtf file to only protein coding, lncRNA, or TCR/Ig genes using the commands in `steps_to_filter_original_gtf.sh`

In [12]:
classical = c('HLA-A', 'HLA-B', 'HLA-C', 'HLA-DRB1', 'HLA-DQA1', 'HLA-DPA1', 'HLA-DPB1', 'HLA-DQB1')

filtered_annot_dir = '../example_data/GRCh38_refs/gencode.v38.annotation.filtered.gtf'
annot = rtracklayer::import(filtered_annot_dir)

masked_annot = annot[which(!(annot$gene_name %in% classical)),]
masked_annot_out = '../example_data/GRCh38_refs/gencode.v38.annotation.filtered.masked.gtf'
rtracklayer::export(masked_annot, masked_annot_out)

# check to make sure this worked
testing = rtracklayer::import(masked_annot_out)
testing[which(testing$gene_name %in% classical),]

GRanges object with 0 ranges and 20 metadata columns:
   seqnames    ranges strand |   source     type     score     phase
      <Rle> <IRanges>  <Rle> | <factor> <factor> <numeric> <integer>
       gene_id   gene_type   gene_name       level     hgnc_id         tag
   <character> <character> <character> <character> <character> <character>
   havana_gene transcript_id transcript_type transcript_name
   <character>   <character>     <character>     <character>
   transcript_support_level havana_transcript exon_number     exon_id
                <character>       <character> <character> <character>
    protein_id      ccdsid
   <character> <character>
  -------
  seqinfo: 25 sequences from an unspecified genome; no seqlengths

`../example_data/GRCh38_refs/gencode.v38.annotation.filtered.masked.gtf` contains the GTF file without any entries for classical HLA genes (ready for concatenating with personalized GTFs)

## All done!

In [13]:
sessionInfo()

R version 4.0.5 (2021-03-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux Server release 6.5 (Santiago)

Matrix products: default
BLAS/LAPACK: /PHShome/jbk37/anaconda3/envs/hla_new/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] forcats_0.5.1        purrr_0.3.4          readr_2.1.2         
 [4] tidyr_1.2.0          tibble_3.1.6         ggplot2_3.3.5       
 [7] tidyverse_1.3.1      stringr_1.4.0        dplyr_1.0.8         
[10] Biostrings_2.58.0    XVector_0.30